In [1]:
from datetime import datetime
import random, time, math
from copy import deepcopy, copy
import decimal

class ChessBoard:
    def __init__(self, size=8):
        self.size = size
        self.reset()

    def reset(self):
        # Initialize queens in random positions
        self.queen_positions = [random.randint(0, self.size - 1) for _ in range(self.size)]

    def calculate_conflicts(self):
        conflicts = 0
        for i in range(self.size):
            for j in range(i + 1, self.size):
                if self._queens_conflict(i, j):
                    conflicts += 1
        return conflicts

    def _queens_conflict(self, q1, q2):
        # Check if two queens are in the same row or on the same diagonal
        return (self.queen_positions[q1] == self.queen_positions[q2] or
                abs(q1 - q2) == abs(self.queen_positions[q1] - self.queen_positions[q2]))

    @staticmethod
    def calculate_conflicts_static(positions):
        conflicts = 0
        size = len(positions)
        for i in range(size):
            for j in range(i + 1, size):
                if (positions[i] == positions[j] or
                    abs(i - j) == abs(positions[i] - positions[j])):
                    conflicts += 1
        return conflicts

    @staticmethod
    def to_string(positions):
        return "\n".join(f"({row}, {col})" for row, col in enumerate(positions))

    def __str__(self):
        return self.to_string(self.queen_positions)

class SimulatedAnnealing:
    def __init__(self, board):
        self.board = board
        self.initial_temperature = 4000
        self.cooling_rate = 0.99
        self.start_time = None
        self.elapsed_time = 0

    def run(self, max_iterations=170000):
        self.start_time = datetime.now()
        current_positions = self.board.queen_positions[:]
        current_conflicts = self.board.calculate_conflicts()
        temperature = self.initial_temperature

        for iteration in range(max_iterations):
            temperature *= self.cooling_rate
            
            # Generate a neighbor solution
            neighbor_positions = current_positions[:]
            i, j = random.sample(range(self.board.size), 2)
            neighbor_positions[i] = j

            # Calculate the change in conflicts
            neighbor_conflicts = ChessBoard.calculate_conflicts_static(neighbor_positions)
            delta_conflicts = neighbor_conflicts - current_conflicts

            # Decide whether to accept the new solution
            if self._should_accept(delta_conflicts, temperature):
                current_positions = neighbor_positions
                current_conflicts = neighbor_conflicts

            # Check if we've found a solution
            if current_conflicts == 0:
                self._print_solution(current_positions)
                return True

        self._print_failure()
        return False

    def _should_accept(self, delta_conflicts, temperature):
        if delta_conflicts < 0:
            return True
        probability = decimal.Decimal(math.e) ** (decimal.Decimal(-delta_conflicts) / decimal.Decimal(temperature))
        return random.random() < probability

    def _print_solution(self, positions):
        print("Solution found:")
        print(ChessBoard.to_string(positions))
        self.elapsed_time = self._get_elapsed_time()
        print(f"Success, Elapsed Time: {self.elapsed_time:.2f}ms")

    def _print_failure(self):
        self.elapsed_time = self._get_elapsed_time()
        # print(f"Unsuccessful, Elapsed Time: {self.elapsed_time:.2f}ms")

    def _get_elapsed_time(self):
        return (datetime.now() - self.start_time).total_seconds() * 1000

if __name__ == '__main__':
    board = ChessBoard()
    print("Initial board configuration:")
    print(board)
    solver = SimulatedAnnealing(board)
    solver.run()

Initial board configuration:
(0, 4)
(1, 0)
(2, 6)
(3, 5)
(4, 5)
(5, 4)
(6, 3)
(7, 3)
Unsuccessful, Elapsed Time: 2471.09ms
